In [1]:
#| default_exp assistant

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.15.0


In [4]:
import torch
print(torch.__version__)

2.1.1


In [11]:
#pd.read_csv('data/009-1.csv')

### Zero-shot Model

In [12]:
#df = pd.read_csv('/Users/hollyhou/Downloads/009-1.csv')
#df.head()

In [5]:
import pandas as pd
import gradio as gr
from transformers import pipeline

# Function to process the uploaded file
def process_transcript(uploaded_file):
    # Read the uploaded file into a DataFrame
    df = pd.read_csv(uploaded_file)

    # Initialize the zero-shot classifier
    classifier = pipeline("zero-shot-classification", model="sileod/deberta-v3-base-tasksource-nli")

    # Define your candidate labels
    candidate_labels = ['PRS', 'REP', 'OTR', 'NEU']

    # Positive and negative words for rule-based classification
    positive_words = ['great', 'well', 'excellent', 'good', 'proud', 'amazing','exactly']
    negative_words = ['bad', 'stop', 'disrespectful', 'quiet', 'get out']

    # Function to apply rule-based override
    def apply_rule_based_override(text, predicted_label, predicted_score):
        text_lower = text.lower()
        if any(word in text_lower for word in positive_words):
            return 'PRS', 0.5  # Assigning full confidence to rule-based predictions
        elif any(word in text_lower for word in negative_words):
            return 'REP', 0.5
        elif text.strip().endswith('?'):
            return 'OTR', 0.5
        else:
            return predicted_label, predicted_score

    # Classify texts and apply the rule-based override
    final_labels = []
    final_scores = []

    for text in df['Text']:
    # Get prediction from the classifier
        prediction = classifier(text, candidate_labels, truncation=True, max_length=1024)
        predicted_label = prediction['labels'][0]  # Top label
        predicted_score = prediction['scores'][0]  # Confidence score for the top label

    # Apply the rule-based override
        final_label, final_score = apply_rule_based_override(text, predicted_label, predicted_score)
        final_labels.append(final_label)
        final_scores.append(final_score)

    # Add the results to the DataFrame
    df['Predicted_Label'] = final_labels
    df['Confidence_Score'] = final_scores

    # Save the processed dataframe to a new CSV file
    output_file = '/tmp/labeled_transcript.csv'
    df.to_csv(output_file, index=False)
    return df, output_file



# Create the Gradio interface
iface = gr.Interface(
    fn=process_transcript,
    inputs=gr.inputs.File(label="Upload Transcript"),
    outputs=[
        gr.outputs.Dataframe(label="Labeled Transcript", type="pandas"),
        gr.outputs.File(label="Download Labeled Transcript")
    ],
    title="Transcript Labeling Tool",
    description="Upload a transcript and get it labeled based on classroom interaction categories."
)

# Run the interface
iface.launch(share=True)

/var/folders/4d/90l81fmn25scytf9lthr56fc0000gn/T/ipykernel_12680/3594426472.py:61: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.File(label="Upload Transcript"),
/var/folders/4d/90l81fmn25scytf9lthr56fc0000gn/T/ipykernel_12680/3594426472.py:61: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.File(label="Upload Transcript"),
/var/folders/4d/90l81fmn25scytf9lthr56fc0000gn/T/ipykernel_12680/3594426472.py:61: GradioDeprecationWarning: `keep_filename` parameter is deprecated, and it has no effect
  inputs=gr.inputs.File(label="Upload Transcript"),
/var/folders/4d/90l81fmn25scytf9lthr56fc0000gn/T/ipykernel_12680/3594426472.py:63: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  gr.outputs.Dataframe(la

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://484e526e57b9674794.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [6]:
#| hide
import nbdev; nbdev.nbdev_export()